In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re


In [57]:
# first, define function to count unique words in a string

def count_unique_words(my_string):
    
    # function that takes a string of words and counts the number 
    # of times each unique word appears
    punctuation = "',."
    no_punc = ""
    # get rid of punctuation
    for i in my_string:
        if i not in punctuation:
            no_punc = no_punc + i
        else:
            no_punc = no_punc
        
    # make a list with each element as a word from string
    # get everything lower case
    words = str.lower(no_punc).split()
    
    # get rid Tiger's name and all words under 4 letters
    unwanted_words = ['tiger', 'woods:']
    words = [word for word in words if word not in unwanted_words]
    words = [word for word in words if len(word) > 3]
    # get list of unique words
    unique_words = []
    for i in words:
        if i not in unique_words:
            unique_words.append(i)

    # count number of times each unique word appears
    word_counter = []
    for i in unique_words:
        count = 0
        for j in words:
            if i == j:
                count += 1
        word_counter.append(count)

    # make DataFrame
    data =  np.column_stack((unique_words, word_counter))
    data2 = pd.DataFrame(data=data, columns = ['word', 'count'])
    return(data2)

In [68]:
r_main = requests.get('http://www.asapsports.com/show_player.php?id=10085')
main_soup = BeautifulSoup(r_main.content, 'html.parser')

final_data = pd.DataFrame()

for interview in main_soup.findAll('table')[-2].findAll('tr')[:5]:
    date = interview.find('nobr').get_text().replace('[ ', '').replace(' ]', '')
    event = interview.find('a').get_text()
    link = interview.find('a')['href']
    print('fetching: ' + date + ' - ' + event + '...')
    
    # use the link to scrape his interview
    r_sub = requests.get(link)
    sub_soup = BeautifulSoup(r_sub.content, 'html.parser')
    # loop through all elements that contain text: 'TIGER WOODS:'
    all_words = ''
    for answer in sub_soup.findAll(text = re.compile('TIGER WOODS:')):
        # get text and replace his name, comma, period
        text = answer.lstrip().replace('TIGER WOODS: ', '').replace(',', '').replace('.', ' ')
        all_words += text
    
    # run function to clean text and count unique words
    data = count_unique_words(all_words)
    # add columns to data frame
    data['date'] = date
    data['event'] = event
    
    final_data = final_data.append(data)
    
print('Done!')

#write to csv
final_data.to_csv('~\Dropbox\tiger_interviews', index= False)




fetching: February 2, 2017 - OMEGA DUBAI DESERT CLASSIC...
fetching: February 1, 2017 - OMEGA DUBAI DESERT CLASSIC...
fetching: October 2, 2016 - THE RYDER CUP...
fetching: August 23, 2015 - WYNDHAM CHAMPIONSHIP...
fetching: August 22, 2015 - WYNDHAM CHAMPIONSHIP...
Done!


In [23]:
r_main = requests.get('http://www.asapsports.com/show_player.php?id=10085')
main_soup = BeautifulSoup(r_main.content, 'html.parser')

all_data = []

# loop through all the interviews (only does 5 for testing)
for interview in main_soup.findAll('table')[-2].findAll('tr')[:5]: # this is only going to do the first 5!!
    # get the date, event, and link
    date = interview.find('nobr').get_text().replace('[ ', '').replace(' ]', '')
    event = interview.find('a').get_text()
    link = interview.find('a')['href']
    print('fetching: ' + date + ' - ' + event + '...')
    
    # use the link to scrape his interview
    r_sub = requests.get(link)
    sub_soup = BeautifulSoup(r_sub.content, 'html.parser')
    # loop through all elements that contain text: 'TIGER WOODS:'
    for answer in sub_soup.findAll(text = re.compile('TIGER WOODS:')):
        new = {}
        # get text and replace his name, comma, period
        text = answer.lstrip().replace('TIGER WOODS: ', '').replace(',', '').replace('.', '')
        # split text into python list of words
        words = text.split()
        
        ####################################
        # DO SHIT HERE TO THE LIST OF WORDS
        ####################################
        
        # fill in dictionary to complete observation
        new['date'] = date
        new['event'] = event
        new['text'] = words
        # finally append this dictionary to the grand list
        all_data.append(new)

print('Done!!')
# turn this list of dictionaries into a pandas df
#all_data = pd.DataFrame(all_data)

fetching: February 2, 2017 - OMEGA DUBAI DESERT CLASSIC...
fetching: February 1, 2017 - OMEGA DUBAI DESERT CLASSIC...
fetching: October 2, 2016 - THE RYDER CUP...
fetching: August 23, 2015 - WYNDHAM CHAMPIONSHIP...
fetching: August 22, 2015 - WYNDHAM CHAMPIONSHIP...
Done!!
